In [167]:
###importing packages
import newspaper
from newspaper import Article
from newspaper import fulltext
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import selenium 
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.chrome.options import Options

In [306]:
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [9]:
###establishing api key
NYT_KEY = %env NYT_KEY 
NYT_SECRET = %env NYT_SECRET

In [290]:
###creating function to pull NYT urls for climate articles
def api_call(i):
    params = {'api-key':NYT_KEY, 'q': 'climate change', 'fq': ['type_of_material = Article', 'source = The New York Times'], 
          'fl' : ['abstract', 'web_url', 'headline', 'pub_date', 'byline', 'keywords', 'word_count'],
          'begin_date': '20221101', 'end_date':'20231031', 'page': i}
    response = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json', params = params)
    nyt_json = response.json()
    nyt_json_df = pd.DataFrame(nyt_json['response']['docs'])
    return nyt_json_df

In [289]:
###seeing how many hits there are
params = {'api-key':NYT_KEY, 'q': 'climate change', 'fq': ['type_of_material = Article', 'source = The New York Times'],
          'fl' : ['abstract', 'web_url', 'source', 'headline', 'pub_date', 'byline', 'keywords', 'word_count', 'document_type'],
          'begin_date': '20221101', 'end_date':'20231031'}
response = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json', params = params)
json = response.json()
json_df = pd.DataFrame(json['response']['docs'])

json['response']['meta']

{'hits': 424, 'offset': 0, 'time': 58}

###My notes
pulled 1-200 pages of articles
need to pull 201-301

In [291]:
dfs = []

In [299]:
###pulling articles for remaining pages of api call 
for i in range(201, 301):
    time.sleep(12)
    dfs.append(api_call(i))

KeyError: 'response'

In [301]:
###checking length of dfs
print(len(dfs))
print(nyt_df.shape)

201
(2010, 7)


In [302]:
###combining df of each api pull
nyt_df = pd.concat(dfs, ignore_index = True)
nyt_df

,abstract,web_url,headline,keywords,pub_date,byline,word_count
0,How anxiety about the planet’s future is trans...,https://www.nytimes.com/2023/10/21/magazine/cl...,{'main': 'Climate Change Is Keeping Therapists...,"[{'name': 'subject', 'value': 'Global Warming'...",2023-10-21T09:02:33+0000,"{'original': 'By Brooke Jarvis', 'person': [{'...",3528
1,Representative Mike Johnson comes from Louisia...,https://www.nytimes.com/2023/10/26/climate/mik...,{'main': 'New House Speaker Champions Fossil F...,"[{'name': 'subject', 'value': 'United States P...",2023-10-26T09:03:55+0000,"{'original': 'By Lisa Friedman', 'person': [{'...",822
2,The container shipping lines that carry the bu...,https://www.nytimes.com/2023/10/30/business/ec...,{'main': 'Shipping Contributes Heavily to Clim...,"[{'name': 'subject', 'value': 'Global Warming'...",2023-10-30T09:00:50+0000,"{'original': 'By Ana Swanson', 'person': [{'fi...",1581
3,Prices have more than doubled as extreme weath...,https://www.nytimes.com/2023/10/22/business/ol...,{'main': 'Why Olive Oil Is So Expensive Right ...,"[{'name': 'subject', 'value': 'Olives', 'rank'...",2023-10-22T09:00:19+0000,"{'original': 'By Santul Nerkar', 'person': [{'...",996
4,By the time the butterflies finally arrived he...,https://www.nytimes.com/2023/10/09/opinion/cli...,{'main': 'What Climate Change Does to My Octob...,"[{'name': 'subject', 'value': 'Flowers and Pla...",2023-10-09T09:01:46+0000,"{'original': 'By Margaret Renkl', 'person': [{...",1210
...,...,...,...,...,...,...,...
2005,"It might sound like all chirps to you, but bir...",https://www.nytimes.com/interactive/2023/07/21...,{'main': 'Can You Understand Bird? Test Your R...,"[{'name': 'subject', 'value': 'internal-paywal...",2023-07-21T12:57:46+0000,"{'original': 'By Elaine Chen', 'person': [{'fi...",0
2006,"Unlike its wealthier neighbors, Fort Myers Bea...",https://www.nytimes.com/2023/09/28/us/hurrican...,"{'main': 'After Hurricane Ian, Who Will Get to...","[{'name': 'subject', 'value': 'Hurricane Ian (...",2023-09-28T09:02:13+0000,{'original': 'By Jennifer Reed and Patricia Ma...,1212
2007,Whimsically chronicling the creation of a Marv...,https://www.nytimes.com/2023/05/07/books/tom-h...,{'main': 'Nice Guy Meets Iron Man in the First...,"[{'name': 'subject', 'value': 'Books and Liter...",2023-05-07T09:00:15+0000,"{'original': 'By Alexandra Jacobs', 'person': ...",983
2008,Walter Isaacson’s biography of the billionaire...,https://www.nytimes.com/2023/09/09/books/revie...,{'main': 'Elon Musk Wants to Save Humanity. Th...,"[{'name': 'persons', 'value': 'Isaacson, Walte...",2023-09-09T16:17:24+0000,"{'original': 'By Jennifer Szalai', 'person': [...",1226


In [342]:
###creating function to pull text from articles using urls pulled from nyt api
def article_download(pulled_url):
    article = Article(url = pulled_url, fetch_images = False, keep_article_html = True)
    article.download()
    article.parse()
    article.clean_top_node
    article_dict = {'text': article.text,
                   'html': article.html}
    return article_dict

In [ ]:
text_df = []

In [346]:
###pulling out article text from every article url
for rows in range(567, 2011):
    text = article_download(nyt_df['web_url'][rows])['text']
    text_df.append(text)

KeyError: 2010

In [381]:
###inserting missing row of text data
missing_text = article_download(nyt_df['web_url'][566])['text']
text_df.insert(566, missing_text)

In [386]:
###adding column within nyt_df containing text
nyt_df['text'] = text_df

In [387]:
nyt_df

,abstract,web_url,headline,keywords,pub_date,byline,word_count,text
0,How anxiety about the planet’s future is trans...,https://www.nytimes.com/2023/10/21/magazine/cl...,{'main': 'Climate Change Is Keeping Therapists...,"[{'name': 'subject', 'value': 'Global Warming'...",2023-10-21T09:02:33+0000,"{'original': 'By Brooke Jarvis', 'person': [{'...",3528,"By 2021, when the Northwest suffered through a..."
1,Representative Mike Johnson comes from Louisia...,https://www.nytimes.com/2023/10/26/climate/mik...,{'main': 'New House Speaker Champions Fossil F...,"[{'name': 'subject', 'value': 'United States P...",2023-10-26T09:03:55+0000,"{'original': 'By Lisa Friedman', 'person': [{'...",822,"Representative Mike Johnson of Louisiana, the ..."
2,The container shipping lines that carry the bu...,https://www.nytimes.com/2023/10/30/business/ec...,{'main': 'Shipping Contributes Heavily to Clim...,"[{'name': 'subject', 'value': 'Global Warming'...",2023-10-30T09:00:50+0000,"{'original': 'By Ana Swanson', 'person': [{'fi...",1581,On a bright September day on the harbor in Cop...
3,Prices have more than doubled as extreme weath...,https://www.nytimes.com/2023/10/22/business/ol...,{'main': 'Why Olive Oil Is So Expensive Right ...,"[{'name': 'subject', 'value': 'Olives', 'rank'...",2023-10-22T09:00:19+0000,"{'original': 'By Santul Nerkar', 'person': [{'...",996,There is something different lately about the ...
4,By the time the butterflies finally arrived he...,https://www.nytimes.com/2023/10/09/opinion/cli...,{'main': 'What Climate Change Does to My Octob...,"[{'name': 'subject', 'value': 'Flowers and Pla...",2023-10-09T09:01:46+0000,"{'original': 'By Margaret Renkl', 'person': [{...",1210,"In Middle Tennessee, October is supposed to me..."
...,...,...,...,...,...,...,...,...
2005,"It might sound like all chirps to you, but bir...",https://www.nytimes.com/interactive/2023/07/21...,{'main': 'Can You Understand Bird? Test Your R...,"[{'name': 'subject', 'value': 'internal-paywal...",2023-07-21T12:57:46+0000,"{'original': 'By Elaine Chen', 'person': [{'fi...",0,Language has long been considered the exclusiv...
2006,"Unlike its wealthier neighbors, Fort Myers Bea...",https://www.nytimes.com/2023/09/28/us/hurrican...,"{'main': 'After Hurricane Ian, Who Will Get to...","[{'name': 'subject', 'value': 'Hurricane Ian (...",2023-09-28T09:02:13+0000,{'original': 'By Jennifer Reed and Patricia Ma...,1212,Melissa Hill has gotten used to the transforme...
2007,Whimsically chronicling the creation of a Marv...,https://www.nytimes.com/2023/05/07/books/tom-h...,{'main': 'Nice Guy Meets Iron Man in the First...,"[{'name': 'subject', 'value': 'Books and Liter...",2023-05-07T09:00:15+0000,"{'original': 'By Alexandra Jacobs', 'person': ...",983,THE MAKING OF ANOTHER MAJOR MOTION PICTURE MAS...
2008,Walter Isaacson’s biography of the billionaire...,https://www.nytimes.com/2023/09/09/books/revie...,{'main': 'Elon Musk Wants to Save Humanity. Th...,"[{'name': 'persons', 'value': 'Isaacson, Walte...",2023-09-09T16:17:24+0000,"{'original': 'By Jennifer Szalai', 'person': [...",1226,"ELON MUSK, by Walter Isaacson\n\nAt various mo..."


In [132]:
###if we want to pull full text using beautiful soup
soup_text = []
def SoupText(url):
    soup = BeautifulSoup(url, 'html.parser')
    text = soup.get_text()
    soup_text.append(text)
    soup_dict = {'soup':soup, 'text': text}
    return 

nyt_df['soup_text'] = soup_text





Why Smaller Storms Are Growing More Fearsome, More Often - The New York Times
  




















Skip to contentSkip to site indexSearch & Section NavigationSection NavigationSEARCHNew York Today’s PaperNew York|With Climate Change, Smaller Storms Are Growing More Fearsome, More Oftenhttps://www.nytimes.com/2023/09/30/nyregion/climate-change-flooding-storms.htmlShare full article384AdvertisementSKIP ADVERTISEMENTSupported bySKIP ADVERTISEMENTWith Climate Change, Smaller Storms Are Growing More Fearsome, More OftenThe Friday storm that produced vast flooding in New York City started out earlier in the week as an unremarkable — if unpredictable — weather system.Share full article384New Yorkers were taken by surprise Friday by the deluge of rain.Credit...Anna Watts for The New York TimesBy Hilary HowardSept. 30, 2023At first, it looked as if New York would simply be grazed by light rain on Friday.David Stark, a meteorologist with the National Oceanic and Atmospheric Administration

In [320]:
###trying other package https://github.com/susannapaoli/web-scraper-nyt
driver = webdriver.Chrome()

In [312]:
def get_body(url):
    driver.get(url)
    article_text = ''
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    paragraph = soup.find_all('p')
    for i in paragraph:
        a = i.get_text()
        if a != 'Advertisement' and a != 'Supported by' and a != 'Send any friend a story' and a != 'As a subscriber, you have 10 gift articles to give each month. Anyone can read what you share.' and not a.startswith("By"):
            article_text += a 
            article_text += " "
    time.sleep(8)
    return article_text

In [313]:
nyt_df['article_body'] = nyt_df.apply(lambda x: get_body(x.web_url), axis=1) 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.159)
  (Driver info: chromedriver=2.38.552518 (183d19265345f54ce39cbb94cf81ba5f15905011),platform=Mac OS X 10.16.0 x86_64)


In [389]:
###saving dataframe to csv file
nyt_df.to_csv("/Users/lisayokocarle/Desktop/Columbia/Fall2023/NLP/FinalProject/qmss-nlp-climate/data/article_data.csv")